## Stop-and-Frisk in New York

Stop and frisk was a policy/program which empowered the police in New York (and other cities) to temporarily detain, question, and at times search civilians and suspects on the street for weapons and other contraband. 
This controversial policy was found unconstitutional since it violates the U.S Constitution’s 4th Amendment prohibition of unreasonable searches and seizures. It was also found to violate the 14th Amendment’s equal protection clause.

The judge also appointed an independent monitor to oversee the implementation of a broad set of court-mandated reforms that followed the litigation. A following NYPD mandate required officers to thoroughly justify the reason for making a stop. While the program is not fully scrapped, the stops dropped drastically in the 2011-2013 time period and continue to drop.

For a historical lens, see: https://en.wikipedia.org/wiki/Stop-and-frisk_in_New_York_City

Details about one of the lawsuits (and a time series): https://www.nyclu.org/en/stop-and-frisk-data

New York kept detailed records of these stops, which allows use to see exactly what happened there. Welcome to invetigative journalism through SparkSQl. Here, we will noly use the 2009 data, when NYPD was heavily using this program to stop its citizens. I invite you to continue the investigation yourself later (some ideas listed at the bottom)...

In [1]:
import pyspark.sql.functions as f
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
# Lets read the csv file and inspect it
# Data from https://www1.nyc.gov/site/nypd/stats/reports-analysis/stopfrisk.page
saf = sqlContext.read.csv("/scratch/siads618f22_class_root/siads618f22_class/shared_data/lecture7/stopAndFrisk2009.csv", header=True)
saf.printSchema()

NameError: name 'sqlContext' is not defined

In [3]:
saf.select("frisked").show()
saf.select("frisked").distinct().show()

saf.registerTempTable("saf")

q1 = sqlContext.sql("SELECT year, pct, crimsusp, arstmade, sumissue, frisked, searched, contrabn, sex, race, age FROM saf")
q1.show()

+-------+
|frisked|
+-------+
|      Y|
|      N|
|      Y|
|      N|
|      Y|
|      Y|
|      N|
|      N|
|      N|
|      N|
|      N|
|      Y|
|      Y|
|      Y|
|      Y|
|      Y|
|      Y|
|      N|
|      Y|
|      Y|
+-------+
only showing top 20 rows

+-------+
|frisked|
+-------+
|      Y|
|      N|
+-------+



/sw/pkgs/arc/spark/spark-3.2.1-bin-hadoop3.2/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(
22/10/13 10:33:50 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----+----+-------------+--------+--------+-------+--------+--------+---+----+---+
| year| pct|     crimsusp|arstmade|sumissue|frisked|searched|contrabn|sex|race|age|
+-----+----+-------------+--------+--------+-------+--------+--------+---+----+---+
| 2009|  67|       FELONY|       N|       N|      Y|       N|       N|  M|   B| 19|
| 2009| 115|         MISD|       N|       N|      N|       N|       N|  M|   B| 24|
| 2009|  84|          FEL|       Y|       N|      Y|       Y|       N|  M|   B| 23|
| 2009|  46|          FEL|       N|       N|      N|       N|       N|  M|   B| 22|
| 2009|  62|      ROBBERY|       N|       N|      Y|       Y|       N|  M|   I| 14|
| 2009|  47|     FEL  CPW|       N|       N|      Y|       N|       N|  M|   B| 20|
| 2009|  50|       FELONY|       N|       N|      N|       N|       N|  M|   W| 61|
| 2009|  30|     TRESPASS|       N|       N|      N|       N|       N|  F|   B| 29|
| 2009|  33|          FEL|       N|       N|      N|       N|       N|  M|  

Let's get some summary stats by race
Here are the race categories:

-	A	ASIAN/PACIFIC ISLANDER
-	B	BLACK
-	I	AMERICAN INDIAN/ALASKAN NATIVE
-	P	BLACK-HISPANIC
-	Q	WHITE-HISPANIC
-	W	WHITE
-	X	UNKNOWN
-	Z	OTHER

Info about the data at: https://www1.nyc.gov/assets/nypd/downloads/zip/analysis_and_planning/stop-question-frisk/SQF-File-Documentation.zip


In [4]:
# Let's see how many stops there are for each race and what is the likelihood of being frisked conditional oon being stopped
# We will only consider demographic groups with sufficient data (> 5000 stops)
q2 = sqlContext.sql("SELECT AVG(cast((frisked == 'Y') as int)) as friskRatio, count(*) as stops, race from saf GROUP BY race HAVING stops > 5000 ORDER BY stops")
q2.show()

+------------------+------+----+
|        friskRatio| stops|race|
+------------------+------+----+
|0.5617593754356615| 14346|   Z|
|0.4975363609379638| 16845|   A|
|0.6199903428295509| 35207|   P|
|0.4610175183298819| 53601|   W|
|0.5976333811996023|144848|   Q|
|0.5723654345789428|310611|   B|
+------------------+------+----+



In [5]:
# The goal of stop and frisk was to stop crime. 
# A good heuristic is whether they find a contraband at the end. 
# Let's see how that success varies by race
q3 = sqlContext.sql('''SELECT CAST(AVG(cast((contrabn == 'Y') as int)) as decimal (10,4)) as contraRatio, 
        CAST(AVG(cast((arstmade == 'Y') as int)) as decimal (10,4)) as arrestRatio, 
        CAST(AVG(cast((frisked == 'Y') as int)) as decimal (10,4)) as friskRatio, count(*) as stops, race from saf 
        GROUP BY race HAVING stops > 5000 ORDER BY stops''')
q3.show()

+-----------+-----------+----------+------+----+
|contraRatio|arrestRatio|friskRatio| stops|race|
+-----------+-----------+----------+------+----+
|     0.0123|     0.0506|    0.5618| 14346|   Z|
|     0.0154|     0.0648|    0.4975| 16845|   A|
|     0.0171|     0.0672|    0.6200| 35207|   P|
|     0.0221|     0.0615|    0.4610| 53601|   W|
|     0.0149|     0.0607|    0.5976|144848|   Q|
|     0.0168|     0.0591|    0.5724|310611|   B|
+-----------+-----------+----------+------+----+



In [6]:
# You can collect the results to create a non-distributed object
q3.collect()
# You can also write to file
q3.rdd.map(lambda i: '\t'.join(str(j) for j in i)) \
    .saveAsTextFile('stopAndFriskSimpleStats')

# Use .rdd to convert DF to rdd
q3rdd = q3.rdd
# Use toDF to convert rdd to df
q3again = q3rdd.toDF() 

# You actually do not need the schema here since the rdd object converted 
# from DF already has the column names

In [7]:
# Ok, let's build up even more. You might say maybe the police stops are driven by overall population?
# Let's account for that

# Population Data - 2010 Census
pop = sqlContext.read.csv("/scratch/siads618f22_class_root/siads618f22_class/shared_data/lecture7/nyc_2010pop_2020precincts.csv", header=True)
pop.registerTempTable('pop')

q4 = sqlContext.sql('''SELECT precinct_2020 as pct, P0020001 as Total_Pop, CAST(P0020005*100/P0020001 as decimal(10,2)) as white,
                    CAST(P0020006*100/P0020001 as decimal(10,2)) as black, CAST(P0020002*100/P0020001 as decimal(10,2)) as hispanic,
                    CAST(P0020008*100/P0020001 as decimal(10,2)) as asian
                    FROM pop
                    ''')
q4 = q4.withColumn('pct', trim(q4.pct))
q4.registerTempTable('precinctPop')
q4.show()

+---+---------+-----+-----+--------+-----+
|pct|Total_Pop|white|black|hispanic|asian|
+---+---------+-----+-----+--------+-----+
|  1|    66679|70.83| 2.97|    7.01|15.78|
|  5|    52568|20.43| 4.37|   11.24|62.22|
|  6|    62226|81.10| 1.95|    6.08| 8.01|
|  7|    56355|23.92| 8.12|   31.90|34.06|
|  9|    76443|51.75| 7.15|   23.93|14.17|
| 10|    50180|61.38| 7.01|   17.58|11.17|
| 13|    93640|69.79| 4.67|    9.15|13.65|
| 14|    20651|56.20| 6.55|   10.64|23.89|
| 17|    79126|76.11| 2.00|    5.81|13.95|
| 18|    54066|60.45| 4.88|   16.43|15.52|
| 19|   208259|81.01| 2.31|    6.64| 8.03|
| 20|   102624|76.86| 3.76|    8.39| 8.73|
| 22|       25| 8.00|64.00|   24.00| 0.00|
| 23|    73106|14.86|25.42|   50.14| 7.42|
| 24|   106460|58.21|11.25|   21.36| 6.55|
| 25|    47405| 7.55|40.10|   47.75| 2.59|
| 26|    49508|38.80|20.52|   24.80|12.45|
| 28|    44781|16.14|55.98|   20.98| 3.51|
| 30|    60685|10.18|27.93|   57.39| 2.26|
| 32|    70942| 5.39|67.37|   22.98| 1.72|
+---+------

In [8]:
# Stop Counts by precinct in 2010
q5 = sqlContext.sql('''SELECT pct, COUNT(*) as stops, 
    CAST(AVG(cast((contrabn == 'Y') as int)) as decimal (10,2)) as contraRatio, 
    CAST(AVG(cast((arstmade == 'Y') as int)) as decimal (10,2)) as arrestRatio, 
    CAST(AVG(cast((frisked == 'Y') as int)) as decimal (10,2)) as friskRatio, 
    CAST(SUM(cast((race == 'B') as int))*100/COUNT(*) as decimal (10,2)) as black_prcntstop, 
    CAST(SUM(cast((race == 'W') as int))*100/COUNT(*) as decimal (10,2)) as white_prcntstop,
    CAST(SUM(cast((race == 'Q') as int))*100/COUNT(*) as decimal (10,2)) as hispanic_prcntstop,
    CAST(SUM(cast((race == 'A') as int))*100/COUNT(*) as decimal (10,2)) as asian_prcntstop
    FROM saf
    WHERE pct IS NOT NULL
    GROUP BY pct
    ORDER BY pct ASC''')
q5 = q5.withColumn('pct', trim(q5.pct))
q5.registerTempTable('precintStops')
q5.show()

+---+-----+-----------+-----------+----------+---------------+---------------+------------------+---------------+
|pct|stops|contraRatio|arrestRatio|friskRatio|black_prcntstop|white_prcntstop|hispanic_prcntstop|asian_prcntstop|
+---+-----+-----------+-----------+----------+---------------+---------------+------------------+---------------+
|  1| 2585|       0.02|       0.09|      0.51|          63.21|          11.99|             15.90|           3.21|
|  5| 2880|       0.02|       0.08|      0.46|          40.31|          10.90|             20.90|          13.99|
|  6| 2325|       0.02|       0.07|      0.44|          56.90|          17.55|             16.69|           2.32|
|  7| 3825|       0.04|       0.10|      0.62|          34.93|          11.76|             37.02|           3.76|
|  9| 5196|       0.04|       0.11|      0.46|          33.20|          19.48|             32.64|           2.73|
| 10| 2760|       0.04|       0.10|      0.49|          46.49|          20.07|          

In [9]:
#Now lets merge the two
q6 = sqlContext.sql('''SELECT precinctPop.pct, precintStops.stops, precinctPop.Total_Pop, precintStops.arrestRatio, 
    precintStops.friskRatio, precintStops.contraRatio,
    precintStops.black_prcntstop, precinctPop.black, precintStops.white_prcntstop, precinctPop.white, 
    precintStops.hispanic_prcntstop, precinctPop.hispanic, precintStops.asian_prcntstop, precinctPop.asian
    FROM precintStops
    JOIN precinctPop 
    ON precintStops.pct  = precinctPop.pct 
    ORDER BY precinctPop.pct ASC''')

q6.registerTempTable('precintSummary')
q6.show()
q6.rdd.map(lambda i: '\t'.join(str(j) for j in i)) \
    .saveAsTextFile('stopAndFriskAndDemoStats')


+---+-----+---------+-----------+----------+-----------+---------------+-----+---------------+-----+------------------+--------+---------------+-----+
|pct|stops|Total_Pop|arrestRatio|friskRatio|contraRatio|black_prcntstop|black|white_prcntstop|white|hispanic_prcntstop|hispanic|asian_prcntstop|asian|
+---+-----+---------+-----------+----------+-----------+---------------+-----+---------------+-----+------------------+--------+---------------+-----+
|  1| 2585|    66679|       0.09|      0.51|       0.02|          63.21| 2.97|          11.99|70.83|             15.90|    7.01|           3.21|15.78|
| 10| 2760|    50180|       0.10|      0.49|       0.04|          46.49| 7.01|          20.07|61.38|             23.19|   17.58|           2.83|11.17|
|100| 3020|    47913|       0.10|      0.61|       0.02|          66.09|24.23|          17.09|55.92|             12.38|   14.95|           1.09| 2.75|
|101| 9159|    67065|       0.09|      0.64|       0.02|          89.73|49.29|           0.95|

There is so much more you can do. Maybe consider computing the success scores per racial group. Or gender. Or age. Or determine how things changed over time. 

NYC had various changes to their stop and frisk program. What were their effects?
